# OpenData REST API

## Libraries and Settings

In [27]:
# Libraries
import os
import requests
import json
import pprint

import pandas as pd


In [28]:
# Package list of the Swiss open data portal
baseUrl = 'https://opendata.swiss/api/3/action/package_show'
id = '?id=fc0430b9-4b27-4ea5-a7fe-5cb1cf6eb142'
callUrl = baseUrl + id

# Make the HTTP request
response = requests.get(callUrl)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK
result = response_dict['result']         # extract all the packages from the response
#pprint.pprint(result)                    # pretty print the list to the screen


# Get the url for the data from the dictionary
data_url = result['resources'][0]['url']
print('Data url:     ' + data_url)

# Print the data format
data_format = result['resources'][0]['format']
print('Data format:  ' + data_format)

df = pd.read_csv(data_url)
df.head()

https://opendata.swiss/api/3/action/package_show?id=fc0430b9-4b27-4ea5-a7fe-5cb1cf6eb142

Data url:     https://data.stadt-zuerich.ch/dataset/bau_whg_mpe_mietpreis_raum_zizahl_gn_jahr_od5161/download/BAU516OD5161.csv
Data format:  CSV


,StichtagDatJahr,StichtagDatMonat,RaumeinheitSort,RaumeinheitLang,GliederungSort,GliederungLang,ZimmerSort,ZimmerLang,GemeinnuetzigSort,GemeinnuetzigLang,...,qu50u,qu75,qu75l,qu75u,qu90,qu90l,qu90u,Domain,Sample1,Sample2
0,2022,2022.04,1,Ganze Stadt,0,Ganze Stadt,2,2 Zimmer,0,Alle Wohnungen,...,1360.0,1712.0,1690.0,1750.0,2155.0,2120.0,2200.0,46692,13488,1714
1,2022,2022.04,1,Ganze Stadt,1,Neubau bis 2 Jahre,2,2 Zimmer,0,Alle Wohnungen,...,2080.0,2227.0,2080.0,2380.0,2490.0,2273.0,2914.0,1498,488,60
2,2022,2022.04,1,Ganze Stadt,2,Neubezug bis 2 Jahre,2,2 Zimmer,0,Alle Wohnungen,...,1560.0,1960.0,1900.0,2040.0,2350.0,2319.0,2449.0,13182,3066,473
3,2022,2022.04,1,Ganze Stadt,3,Bestand Mietverträge 2–10 Jahre,2,2 Zimmer,0,Alle Wohnungen,...,1405.0,1701.0,1682.0,1758.0,2099.0,2036.0,2160.0,20055,5902,753
4,2022,2022.04,1,Ganze Stadt,4,Bestand Mietverträge 11-20 Jahre,2,2 Zimmer,0,Alle Wohnungen,...,1160.0,1313.0,1267.0,1390.0,1650.0,1532.0,1819.0,6782,2343,249
